## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
from sklearn.feature_selection import r_regression as pearson_correlation
from sklearn.decomposition import NMF

## Chargement des données : matrice de scoring

In [3]:
orders = pd.read_json("../raw_data/order_scoring.json",lines=True)

In [4]:
counted_sku_df = orders.groupby('sku').count()['LO'].copy()
counted_sku_df.sort_values(ascending=False,inplace=True)
sku_lenght = 500
short_sku_list = list(counted_sku_df.head(sku_lenght).index)
orders_short = orders[orders['sku'].isin(short_sku_list)]

In [5]:
coef = {
    1:0.4,
    2:0.3,
    3:0.2,
    4:0.1
    }
orders_short['score'] = orders_short['LO'].map(coef)
table_scoring = orders_short.groupby(by=['customer_id','sku']).sum()[['score']]

/tmp/ipykernel_2686117/1351466278.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_short['score'] = orders_short['LO'].map(coef)


In [6]:
penality = 0.5
table_penality = pd.pivot_table(orders_short, 
                       values=['score'], 
                       index=['customer_id','sku'],
                       columns=['LO'],
                       aggfunc='count',
                       fill_value=0)

table_penality['penality'] = table_penality[[('score',4),('score',3)]].max(axis=1)*(1-table_penality[('score',2)])*(1-table_penality[('score',1)])*penality
final_table = pd.DataFrame(table_scoring['score'] - table_penality['penality'],columns=["score"])

In [7]:
matrix = pd.pivot_table(final_table, values=['score'], index=['customer_id'],
                    columns=['sku'],aggfunc='sum')
matrix = matrix.fillna(0)

In [8]:
matrix.columns = matrix.columns.droplevel(0)
data = matrix

In [9]:
data

sku,1-ABB-101,1-ABO-103,1-ABO-105,1-ACN-106,1-ACN-107,1-ACN-108,1-ACN-113,1-ACN-116,1-ACP-101,1-ACP-102,...,8-GRI-104,8-ILB-103,8-ILB-106,8-LCM-119,8-RES-102,8-RES-106,8-VLF-104,8-VLF-105,8-VLF-107,8-VLF-108
customer_id,,,,,,,,,,,,,,,,,,,,,
00HCaCqtG9aMGVCDyBUbvzIzY073,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00KfpQoUiRTi8RfGFcdjFM80s5s1,0.0,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00RZel3ipuTJMrADRnZ3Lr1C1XX2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00V8dG1xBHM84aVChJyEgIL21XK2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00zoJSpuLHUBtthekFUQ3sjnJy63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzqXh3hccWTjkxco7Qdz1iEy7bu1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzqaeuDHkFcnofZkDUHPJ7fbCS33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzrZzibp5CbzXtjld4reTvX7bdR2,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
matrix_positive = pd.pivot_table(final_table, values=['score'], index=['customer_id'],
                    columns=['sku'],aggfunc='sum')

In [11]:
matrix_positive = matrix_positive + 1
matrix_positive = matrix_positive.fillna(0)
matrix_positive.columns = matrix_positive.columns.droplevel(0)
data_positive = matrix_positive

## Non negative matrice factorization recommandation model

In [12]:
# On sépare entre train et apply, pour gagner du temps
def person_based_recommandation_nnf_train(data, n_categories=5):
    model = NMF(n_components=n_categories, init='random', max_iter=2000, random_state=0)
    W = model.fit_transform(data)
    H = model.components_
    approximated_matrix = np.matmul(W, H)
    approximated_matrix_df = pd.DataFrame(approximated_matrix, index=data.index, columns=data.columns)
    return approximated_matrix_df

def person_based_recommandation_nnf_apply(data, approximated_matrix_df, utilisateur, n_recommandations=5):
    # Pour recommander un item, on prend le mieux noté parmi ceux qu'il n'avait pas notés.
    data_utilisateur = data.filter(items=[utilisateur], axis=0).transpose()
    data_utilisateur['has_rated'] = (data_utilisateur[utilisateur] != 0)
    
    predicted_ratings_utilisateur = approximated_matrix_df.filter(items=[utilisateur], axis=0).transpose()

    # Un peu pareil que pour Pearson, on prend les meilleurs de ceux qu'on a pas encore notés
    items_sorted = predicted_ratings_utilisateur[data_utilisateur['has_rated'] == False].sort_values(utilisateur, ascending=False)
    return items_sorted[:n_recommandations].index        


In [13]:
# Une fonction qui vient à la fin, après le NMF et la prédiction des scores
# Elle fait la partie à la fin, elle prend tous les scores prédits pour tous les items, et elle renvoie une
# prediction des 3 top items
def recommandations_from_predicted_scores(test_data, predictions, utilisateur, n_recommandations=3):
    approximated_matrix_df = pd.DataFrame(predictions, index=test_data.index, columns=test_data.columns)
    
    # Pour recommander un item, on prend le mieux noté parmi ceux qu'il n'avait pas notés.
    data_utilisateur = data.filter(items=[utilisateur], axis=0).transpose()
    data_utilisateur['has_rated'] = (data_utilisateur[utilisateur] != 0)

    predicted_ratings_utilisateur = approximated_matrix_df.filter(items=[utilisateur], axis=0).transpose()

    # Un peu pareil que pour Pearson, on prend les meilleurs de ceux qu'on a pas encore notés
    items_sorted = predicted_ratings_utilisateur[data_utilisateur['has_rated'] == False].sort_values(utilisateur, ascending=False)
    return items_sorted[:n_recommandations].index

## Tests

In [14]:
customer_id = "h6IvomPZxOTKuKGuvS8b3nZrzVk2"

In [15]:

person_based_recommandation(data, customer_id, n_recommandations=3)

NameError: name 'person_based_recommandation' is not defined

In [16]:

# https://lafourche.fr/products/philia-bouillon-de-legumes-en-poudre-bio-125g-bio
# https://lafourche.fr/products/lot-de-12-compotes-cool-fruits-pomme-acerola
# https://lafourche.fr/products/la-fourche-tomates-concassees-bio-0-4kg

In [17]:
person_based_recommandation_nnf_apply(data_positive, nnf_model, customer_id, n_recommandations=3)

NameError: name 'nnf_model' is not defined

In [ ]:
# https://lafourche.fr/products/elibio-mais-doux-bio-300g
# https://lafourche.fr/products/la-fourche-1kg-de-pates-cocciolette-blanches-bio-en-vrac
# https://lafourche.fr/products/papeco-essuie-tout-blanc-100prct-recycle-origine-france-200-feuilles-ecologique

In [18]:
customer_id = "3630465646727"

In [19]:

person_based_recommandation(data, customer_id, n_recommandations=3)

NameError: name 'person_based_recommandation' is not defined

In [20]:
#https://lafourche.fr/products/la-fourche-puree-pommes-mangues-bio-0-915kg
#https://lafourche.fr/products/vrac-1kg-de-sucre-de-coco-en-vrac-bio
#https://lafourche.fr/products/la-fourche-tomates-concassees-bio-0-4kg

In [21]:
person_based_recommandation_nnf_apply(data_positive, nnf_model, customer_id, n_recommandations=3)

NameError: name 'nnf_model' is not defined

In [ ]:
#https://lafourche.fr/products/elibio-mais-doux-bio-300g
#https://lafourche.fr/products/la-fourche-puree-100-cacahuetes-bio-0-5kg
#https://lafourche.fr/products/la-fourche-1kg-de-pates-cocciolette-blanches-bio-en-vrac

In [22]:
customer_id = "2074102857788"

In [23]:

person_based_recommandation(data, customer_id, n_recommandations=3)

NameError: name 'person_based_recommandation' is not defined

In [24]:
person_based_recommandation_nnf_apply(data_positive, nnf_model, customer_id, n_recommandations=3)

NameError: name 'nnf_model' is not defined

## Grid search on NMF

### Une fonction de scoring

In [86]:
flattened_data = data_positive.stack()  # On transforme la matrice de scoring en une seule colonne
non_zero_data = flattened_data[flattened_data != 0]  # On supprime les zéros

In [87]:
flat_predicted_scores = predicted_scores.stack()
common_index = flat_predicted_scores.index.intersection(non_zero_data.index)

In [63]:
display(non_zero_data[common_index])
display(flat_predicted_scores[common_index])

customer_id                   sku      
VbZIqPUHqvYf65NRUvMKPVndrgG2  1-CLN-109    0.6
                              1-NTV-163    0.6
                              4-BOF-113    0.7
                              4-BOF-124    1.5
                              4-BOF-125    0.7
                                          ... 
s5RVBtqb6OO5ho49vpedjWQ0zgc2  6-EVE-104    0.6
                              6-GAB-101    1.7
                              6-NAT-102    1.9
                              6-VDG-108    0.6
                              6-VDG-109    1.7
Length: 433898, dtype: float64

customer_id                   sku      
VbZIqPUHqvYf65NRUvMKPVndrgG2  1-CLN-109    0.022132
                              1-NTV-163    0.022718
                              4-BOF-113    0.390458
                              4-BOF-124    1.605971
                              4-BOF-125    0.498771
                                             ...   
s5RVBtqb6OO5ho49vpedjWQ0zgc2  6-EVE-104    0.257326
                              6-GAB-101    0.322430
                              6-NAT-102    0.157645
                              6-VDG-108    0.139493
                              6-VDG-109    0.233479
Length: 433898, dtype: float64

In [66]:
(non_zero_data[common_index] - flat_predicted_scores[common_index]).abs().mean()

0.9535187497683372

In [89]:
def non_zero_mean_squared_error(predicted_scores, y=None):
    # On transforme la matrice de scoring en une seule colonne
    flattened_data = data_positive.stack()
    
    # On supprime les zéros
    non_zero_data = flattened_data[flattened_data != 0]
        
    # On transforme la matrice des scores prédits en une seule colonne aussi
    flat_predicted_scores = predicted_scores.stack()
    
    # On ne garde que les indices qui sont dans les deux (la matrice de scoring de départ + la matrice des prédictions)
    common_index = flat_predicted_scores.index.intersection(non_zero_data.index)
    
    # On calcule le mean squared error sur ces index seulement
    error = non_zero_data[common_index] - flat_predicted_scores[common_index]
    return (error * error).mean()

In [108]:
def non_zero_mean_absolute_error(predicted_scores, y=None):
    # On transforme la matrice de scoring en une seule colonne
    flattened_data = data_positive.stack()
    
    # On supprime les zéros
    non_zero_data = flattened_data[flattened_data != 0]
        
    # On transforme la matrice des scores prédits en une seule colonne aussi
    flat_predicted_scores = predicted_scores.stack()
    
    # On ne garde que les indices qui sont dans les deux (la matrice de scoring de départ + la matrice des prédictions)
    common_index = flat_predicted_scores.index.intersection(non_zero_data.index)
    
    # On calcule le mean absolute error sur ces index seulement
    error = non_zero_data[common_index] - flat_predicted_scores[common_index]
    result = error.abs().mean()
    return result

### Une fonction qui marche avec GridSearchCV et train

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [26]:
train_data, test_data = train_test_split(data_positive, test_size=0.3)

In [103]:
class NMFPredictor(NMF):
    # Je rajoute une fonction predict() sur le NMF pour pouvoir l'utiliser et faire des predictions directement
    # dans un pipeline
    def predict(self, test_data):
        W = self.transform(test_data)
        H = self.components_
        
        approximated_matrix = np.matmul(W, H)
        approximated_matrix_df = pd.DataFrame(approximated_matrix, index=test_data.index, columns=test_data.columns)
        return approximated_matrix_df
    
    def score(self, test_data, y=None):
        predicted_scores = self.predict(test_data)
        return - non_zero_mean_absolute_error(predicted_scores)
        
def train_and_predict(train_data, test_data, n_categories=15):
    model = NMFPredictor(n_components=n_categories, init='random', max_iter=2000, random_state=0)
    model.fit(train_data)
    return model.predict(test_data)
    
predicted_scores = train_and_predict(train_data, test_data)

In [28]:
clean_test_data = test_data[test_data != 0]
clean_test_data

sku,1-ABB-101,1-ABO-103,1-ABO-105,1-ACN-106,1-ACN-107,1-ACN-108,1-ACN-113,1-ACN-116,1-ACP-101,1-ACP-102,...,8-GRI-104,8-ILB-103,8-ILB-106,8-LCM-119,8-RES-102,8-RES-106,8-VLF-104,8-VLF-105,8-VLF-107,8-VLF-108
customer_id,,,,,,,,,,,,,,,,,,,,,
VbZIqPUHqvYf65NRUvMKPVndrgG2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2962245091463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OMRp6av1UCY1oB4IgSaZUEXcww43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eTjqhn8qK1dkEz5BiUyvZyQifaE2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3,1.3
jZ057h41A8gyLS2bzCFe9uky2Gs2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191947829308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.6,0.6,NaN,NaN
3414086811783,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.4,NaN,...,NaN,1.3,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KxyUfDGB3xRlU6iyM0RKi9AhrEp2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
mean_squared_error(test_data, predicted_scores)

0.058953273815680995

In [91]:
non_zero_mean_squared_error(predicted_scores)

1.1121493735298813

In [68]:
mean_absolute_error(test_data, predicted_scores)

0.0909645296999995

In [92]:
non_zero_mean_absolute_error(predicted_scores)

0.9535187497683372

In [76]:
predicted_scores

sku,1-ABB-101,1-ABO-103,1-ABO-105,1-ACN-106,1-ACN-107,1-ACN-108,1-ACN-113,1-ACN-116,1-ACP-101,1-ACP-102,...,8-GRI-104,8-ILB-103,8-ILB-106,8-LCM-119,8-RES-102,8-RES-106,8-VLF-104,8-VLF-105,8-VLF-107,8-VLF-108
customer_id,,,,,,,,,,,,,,,,,,,,,
VbZIqPUHqvYf65NRUvMKPVndrgG2,0.014020,0.032537,0.028944,0.000000,0.023728,0.000000,0.000000,0.001204,0.000000,0.000000,...,0.000000,0.004886,0.003718,0.013000,0.002599,0.000000,0.000000,0.000000,0.000000,0.000421
2962245091463,0.017171,0.039093,0.027839,0.007080,0.014963,0.003284,0.008742,0.001123,0.008486,0.010328,...,0.003218,0.011811,0.009308,0.002141,0.007970,0.001601,0.003479,0.002553,0.002928,0.002401
OMRp6av1UCY1oB4IgSaZUEXcww43,0.038239,0.127451,0.082760,0.011786,0.055421,0.025231,0.019377,0.035082,0.173422,0.060101,...,0.064472,0.048028,0.041877,0.058378,0.041272,0.034025,0.017076,0.008029,0.041074,0.030372
eTjqhn8qK1dkEz5BiUyvZyQifaE2,0.015033,0.031386,0.025836,0.026094,0.014277,0.025700,0.037975,0.012496,0.205116,0.185716,...,0.075832,0.077462,0.086103,0.040036,0.086126,0.086397,0.078771,0.080006,0.116174,0.083370
jZ057h41A8gyLS2bzCFe9uky2Gs2,0.004059,0.004972,0.010501,0.000149,0.004625,0.000291,0.000247,0.000538,0.000000,0.014486,...,0.014960,0.003348,0.004449,0.004536,0.002991,0.007326,0.013258,0.007854,0.005739,0.001911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191947829308,0.015610,0.045792,0.023871,0.036552,0.021180,0.039647,0.046348,0.018154,0.211414,0.137667,...,0.066941,0.070556,0.075868,0.047298,0.084007,0.070531,0.058091,0.064934,0.100361,0.074308
3414086811783,0.035152,0.043771,0.027903,0.086802,0.065497,0.147560,0.147480,0.092691,0.404637,0.384525,...,0.092806,0.095574,0.081205,0.089071,0.090877,0.099396,0.085285,0.069146,0.118952,0.076205
KxyUfDGB3xRlU6iyM0RKi9AhrEp2,0.012999,0.002117,0.006223,0.039672,0.014077,0.053156,0.066010,0.027571,0.150052,0.167167,...,0.037694,0.047944,0.049177,0.019116,0.054061,0.059948,0.050255,0.049349,0.070337,0.048250


In [77]:
pd.DataFrame(predicted_scores, index=test_data.index, columns=test_data.columns)

sku,1-ABB-101,1-ABO-103,1-ABO-105,1-ACN-106,1-ACN-107,1-ACN-108,1-ACN-113,1-ACN-116,1-ACP-101,1-ACP-102,...,8-GRI-104,8-ILB-103,8-ILB-106,8-LCM-119,8-RES-102,8-RES-106,8-VLF-104,8-VLF-105,8-VLF-107,8-VLF-108
customer_id,,,,,,,,,,,,,,,,,,,,,
VbZIqPUHqvYf65NRUvMKPVndrgG2,0.014020,0.032537,0.028944,0.000000,0.023728,0.000000,0.000000,0.001204,0.000000,0.000000,...,0.000000,0.004886,0.003718,0.013000,0.002599,0.000000,0.000000,0.000000,0.000000,0.000421
2962245091463,0.017171,0.039093,0.027839,0.007080,0.014963,0.003284,0.008742,0.001123,0.008486,0.010328,...,0.003218,0.011811,0.009308,0.002141,0.007970,0.001601,0.003479,0.002553,0.002928,0.002401
OMRp6av1UCY1oB4IgSaZUEXcww43,0.038239,0.127451,0.082760,0.011786,0.055421,0.025231,0.019377,0.035082,0.173422,0.060101,...,0.064472,0.048028,0.041877,0.058378,0.041272,0.034025,0.017076,0.008029,0.041074,0.030372
eTjqhn8qK1dkEz5BiUyvZyQifaE2,0.015033,0.031386,0.025836,0.026094,0.014277,0.025700,0.037975,0.012496,0.205116,0.185716,...,0.075832,0.077462,0.086103,0.040036,0.086126,0.086397,0.078771,0.080006,0.116174,0.083370
jZ057h41A8gyLS2bzCFe9uky2Gs2,0.004059,0.004972,0.010501,0.000149,0.004625,0.000291,0.000247,0.000538,0.000000,0.014486,...,0.014960,0.003348,0.004449,0.004536,0.002991,0.007326,0.013258,0.007854,0.005739,0.001911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191947829308,0.015610,0.045792,0.023871,0.036552,0.021180,0.039647,0.046348,0.018154,0.211414,0.137667,...,0.066941,0.070556,0.075868,0.047298,0.084007,0.070531,0.058091,0.064934,0.100361,0.074308
3414086811783,0.035152,0.043771,0.027903,0.086802,0.065497,0.147560,0.147480,0.092691,0.404637,0.384525,...,0.092806,0.095574,0.081205,0.089071,0.090877,0.099396,0.085285,0.069146,0.118952,0.076205
KxyUfDGB3xRlU6iyM0RKi9AhrEp2,0.012999,0.002117,0.006223,0.039672,0.014077,0.053156,0.066010,0.027571,0.150052,0.167167,...,0.037694,0.047944,0.049177,0.019116,0.054061,0.059948,0.050255,0.049349,0.070337,0.048250


In [33]:
# Un customer au hasard dans test_data
test_customer_id = test_data.index[0]
test_customer_id

'VbZIqPUHqvYf65NRUvMKPVndrgG2'

In [ ]:
# Une fonction qui vient à la fin, après le NMF et la prédiction des scores
# Elle fait la partie à la fin, elle prend tous les scores prédits pour tous les items, et elle renvoie une
# prediction des 3 top items
def recommandations_from_predicted_scores(test_data, predictions, utilisateur, n_recommandations=3):
    approximated_matrix_df = pd.DataFrame(predictions, index=test_data.index, columns=test_data.columns)
    
    # Pour recommander un item, on prend le mieux noté parmi ceux qu'il n'avait pas notés.
    data_utilisateur = data.filter(items=[utilisateur], axis=0).transpose()
    data_utilisateur['has_rated'] = (data_utilisateur[utilisateur] != 0)

    predicted_ratings_utilisateur = approximated_matrix_df.filter(items=[utilisateur], axis=0).transpose()

    # Un peu pareil que pour Pearson, on prend les meilleurs de ceux qu'on a pas encore notés
    items_sorted = predicted_ratings_utilisateur[data_utilisateur['has_rated'] == False].sort_values(utilisateur, ascending=False)
    return items_sorted[:n_recommandations].index        

recommandations_from_predicted_scores(test_data, predicted_scores, test_customer_id)

In [34]:
recommandations_from_predicted_scores(test_data, predicted_scores, test_customer_id)

Index(['4-BOF-131', '4-BOF-126', '4-LAF-126'], dtype='object', name='sku')

### Grid Search

In [71]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer

In [112]:
param_grid = {
    # 'nmf__n_components': [5, 25, 50, 75, 100, 200, 300]
    'n_components': [5, 25, 50, 75, 100, 150, 200, 250, 300]
}

nmf_predictor = NMFPredictor(init='random', max_iter=2000, random_state=0)

grid_search = GridSearchCV(nmf_predictor, param_grid, cv=5, verbose=3)
grid_search.fit(data_positive)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ...................n_components=5;, score=-1.027 total time=   9.1s
[CV 2/5] END ...................n_components=5;, score=-1.022 total time=   9.2s
[CV 3/5] END ...................n_components=5;, score=-1.015 total time=   9.8s
[CV 4/5] END ...................n_components=5;, score=-1.029 total time=  10.2s
[CV 5/5] END ...................n_components=5;, score=-1.018 total time=  11.0s
[CV 1/5] END ..................n_components=25;, score=-0.916 total time=  36.6s
[CV 2/5] END ..................n_components=25;, score=-0.911 total time=  21.7s
[CV 3/5] END ..................n_components=25;, score=-0.889 total time=  46.9s
[CV 4/5] END ..................n_components=25;, score=-0.903 total time=  22.7s
[CV 5/5] END ..................n_components=25;, score=-0.889 total time=  33.0s
[CV 1/5] END ..................n_components=50;, score=-0.817 total time=  28.5s
[CV 2/5] END ..................n_components=50;, 

GridSearchCV(cv=5,
             estimator=NMFPredictor(init='random', max_iter=2000,
                                    random_state=0),
             param_grid={'n_components': [5, 25, 50, 75, 100, 150, 200, 250,
                                          300]},
             verbose=3)

In [113]:
grid_search.best_estimator_

NMFPredictor(init='random', max_iter=2000, n_components=300, random_state=0)